# Imports, Functions, Global Variables

In [281]:
#!pip install requests
import requests
import json
import pandas as pd
import numpy as np
from copy import Error
from datetime import datetime
from datetime import timedelta

from google.colab import drive
drive.mount('/content/drive')

def float_to_string(f):
  f = np.round(f,4)
  s = str(f)[2:]
  while len(s) < 4:
    s += "0"
  return s

def pad_date(s):
  if len(s) < 4:
    s = "0" + s
  return s

team_name_dict = {
 'Montreal': 'Montréal Canadiens',
 'Toronto': 'Toronto Maple Leafs',
 'Boston': 'Boston Bruins',
 'Washington': 'Washington Capitals',
 'Calgary': 'Calgary Flames',
 'Vancouver': 'Vancouver Canucks',
 'Anaheim': 'Anaheim Ducks',
 'SanJose': 'San Jose Sharks',
 'Pittsburgh': 'Pittsburgh Penguins',
 'NYIslanders': 'New York Islanders',
 'Carolina': 'Carolina Hurricanes',
 'Buffalo': 'Buffalo Sabres',
 'Columbus': 'Columbus Blue Jackets',
 'Detroit': 'Detroit Red Wings',
 'Chicago': 'Chicago Blackhawks',
 'Ottawa': 'Ottawa Senators',
 'Nashville': 'Nashville Predators',
 'NYRangers': 'New York Rangers',
 'Winnipeg': 'Winnipeg Jets',
 'St.Louis': 'St. Louis Blues',
 'Arizona': 'Arizona Coyotes',
 'Dallas': 'Dallas Stars',
 'Minnesota': 'Minnesota Wild',
 'Colorado': 'Colorado Avalanche',
 'Philadelphia': 'Philadelphia Flyers',
 'Vegas': 'Vegas Golden Knights',
 'LosAngeles': 'Los Angeles Kings',
 'Edmonton': 'Edmonton Oilers',
 'NewJersey': 'New Jersey Devils',
 'Florida': 'Florida Panthers',
 'TampaBay': 'Tampa Bay Lightning'
}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Get betting data for upcoming games

In [ ]:
import requests


# An api key is emailed to you when you sign up to a plan
# Get a free API key at https://api.the-odds-api.com/
API_KEY = 'f700f25d69f3edff31db5babf6b72141'

SPORT = 'icehockey_nhl' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

REGIONS = 'us' # uk | us | eu | au. Multiple can be specified if comma delimited

MARKETS = 'h2h,spreads,totals' # h2h | spreads | totals. Multiple can be specified if comma delimited

ODDS_FORMAT = 'decimal' # decimal | american

DATE_FORMAT = 'iso' # iso | unix

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
#
# First get a list of in-season sports
#   The sport 'key' from the response can be used to get odds in the next request
#
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

'''
sports_response = requests.get(
    'https://api.the-odds-api.com/v4/sports', 
    params={
        'api_key': API_KEY
    }
)


if sports_response.status_code != 200:
    print(f'Failed to get sports: status_code {sports_response.status_code}, response body {sports_response.text}')

else:
    print('List of in season sports:', sports_response.json())
'''


# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
#
# Now get a list of live & upcoming games for the sport you want, along with odds for different bookmakers
# This will deduct from the usage quota
# The usage quota cost = [number of markets specified] x [number of regions specified]
# For examples of usage quota costs, see https://the-odds-api.com/liveapi/guides/v4/#usage-quota-costs
#
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
)

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()
    print('Number of events:', len(odds_json))
    print(odds_json)

    # Check the usage quota
    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])


# Hockey Objects

In [325]:
class HockeySeason:

  def __init__(self, season_string):

# Get dates from the master schedule
    response = requests.get(f"https://statsapi.web.nhl.com/api/v1/schedule?season={season_string}")
    js = json.loads(response.content.decode('utf-8'))

    season = pd.DataFrame(columns = ['game_id', 'date', 'game_day'])

    for i in range(len(js['dates'])):
      date = js['dates'][i]['date']
      for j in range(len(js['dates'][i]['games'])):
        game_id = str(js['dates'][i]['games'][j]['gamePk'])
        game_type = js['dates'][i]['games'][j]['gameType']
        data = {'game_id': [game_id], 'date': [date], 'game_day': [i], 'game_type':[game_type]}
        season = pd.concat([season,pd.DataFrame(data)])

# Get info from boxscores, player by player
    end_reached = False
    self.year = season_string[:4]
    game_number = .0001
    list_season = []

    while not end_reached:

      try:
        game_string = float_to_string(game_number)
        boxscore_string = f"https://statsapi.web.nhl.com/api/v1/game/{self.year}02{game_string}/boxscore"
        boxscore_response = requests.get(boxscore_string)
      except:
        #Reached the end of the season
        end_reached = True
        break

      game = json.loads(boxscore_response.content.decode('utf-8'))
      teams=['away', 'home']

      for team in teams:
        try:
          players = list(game['teams'][team]['players'].keys())
        except:
          end_reached = True
          break
        stats = []
        for player in players:
          try:
            game['teams'][team]['players'][player]['stats']['skaterStats']['player_id'] = player
            stats.append(game['teams'][team]['players'][player]['stats']['skaterStats'])
          except:
            try:
              #Player is a goalie or is scratched
              game['teams'][team]['players'][player]['stats']['goalieStats']['player_id'] = player
              stats.append(game['teams'][team]['players'][player]['stats']['goalieStats'])
            except:
              #Player scratched
              continue
        df = pd.DataFrame.from_dict(stats)
        df = df.set_index('player_id')
        df[f'{team}_team_name'] = game['teams'][team]['team']['name']
        df[f'{team}_team_id'] = game['teams'][team]['team']['id']
        df['season'] = year
        df['game_number'] = game_number
        df['game_id'] = f"{year}02{game_string}"
        list_season.append(df)

      game_number += .0001

    self.boxscore_df = pd.concat(list_season)
    self.schedule_df = season[season['game_type'] == 'R']

  def process_box_scores(self):

    self.boxscore_df['game_number'] = self.boxscore_df['game_number'] * 10000
    self.boxscore_df['game_number'] = self.boxscore_df['game_number'].astype(int)

    home_teams = self.boxscore_df[self.boxscore_df['away_team_name'].isna()]
    away_teams = self.boxscore_df[self.boxscore_df['home_team_name'].isna()]
    by_game_away = away_teams.groupby(['game_id']).sum()
    by_game_home = home_teams.groupby(['game_id']).sum()
    num_games = len(by_game_home)

    if len(by_game_away) != num_games:
      print("Error: Number of home and away games don't match")
      print(f"Home: {num_games}")
      print(f"Away: {len(by_game_away)}")
      return False

    season = []

    for i in range(1,num_games+1,1):
      box_score = {}
      this_game_all = self.boxscore_df[self.boxscore_df['game_number'] == i]
      this_game_home = this_game_all[this_game_all['away_team_name'].isna()]
      this_game_away = this_game_all[this_game_all['home_team_name'].isna()]
      box_score['game_number'] = i
      try:
        box_score['home_team_name'] = this_game_home.home_team_name.iloc[0]
        box_score['away_team_name'] = this_game_away.away_team_name.iloc[0]
        box_score['game_id'] = this_game_home.game_id.iloc[0]
      except:
        print(f"Error getting the team names or game ID in game {i}")
      #print(by_game_home)
      try:
        box_score['home_goals'] = this_game_home.goals.sum()
        box_score['away_goals'] = this_game_away.goals.sum()
        box_score['home_shots'] = this_game_home[this_game_home.evenSaves.isna()].shots.sum()
        box_score['away_shots'] = this_game_away[this_game_away.evenSaves.isna()].shots.sum()
        box_score['home_hits'] = this_game_home.hits.sum()
        box_score['away_hits'] = this_game_away.hits.sum()
        box_score['home_ppg'] = this_game_home.powerPlayGoals.sum()
        box_score['away_ppg'] = this_game_away.powerPlayGoals.sum()
        box_score['home_goals'] = this_game_home.goals.sum()
        box_score['away_goals'] = this_game_away.goals.sum()

      except:
        print("Error getting goals")
        print()
        return False
      season.append(box_score)

    self.processed_boxscores = pd.DataFrame(season)

    self.processed_boxscores['periods_played'] = 0

    for i in range(len(self.processed_boxscores)):
      game_string = self.processed_boxscores.loc[i].game_id
      linescore_string = f"https://statsapi.web.nhl.com/api/v1/game/{game_string}/linescore"
      linescore_response = requests.get(linescore_string)
      linescore = json.loads(linescore_response.content.decode('utf-8'))
      try:
        num_periods = int(linescore['currentPeriod'])
      except:
        print(linescore)
      self.processed_boxscores.at[i, 'periods_played'] = num_periods


    self.full_season = self.processed_boxscores.set_index('game_id').join(self.schedule_df.set_index('game_id'), how='right')

class HockeySeasonOdds():

  def __init__(self, link, team_names):

    odds = pd.read_csv(link)

    cols = ['date', 'home_team_name', 'away_team_name', 'first_period_home', 'second_period_home', 'third_period_home',
                                'first_period_away', 'second_period_away', 'third_period_away', 'final_score',
                                'opening_moneyline_home', 'closing_moneyline_home', 'opening_moneyline_away', 'closing_moneyline_away',
                                'spread', 'away_spread_odds', 'home_spread_odds', 'open_OU', 'close_OU', 'over_odds', 'under_odds']

    self.final_odds = pd.DataFrame(columns = cols)

    odds.index = np.arange(1, len(odds)+1)
    game_number = 1
    for index, row in odds.iterrows():

      if index % 2 == 0:
        result = {}

        d = datetime.strptime(pad_date(str(row['Date'])), "%m%d")
        date = f"{d.month}/{d.day}"
        result['date'] = date
        result['month'] = int(d.month)
        result['day'] = int(d.day)

        result['game_number'] = game_number
        game_number += 1

        home_team = row['Team']
        away_team = odds.loc[index-1]['Team']



        result['home_team_name'] = team_name_dict[home_team]
        result['away_team_name'] = team_name_dict[away_team]
        result['first_period_home'] = row['1st']
        result['second_period_home'] = row['2nd']
        result['third_period_home'] = row['3rd']
        result['first_period_away'] = odds.loc[index-1]['1st']
        result['second_period_away'] = odds.loc[index-1]['2nd']
        result['third_period_away'] = odds.loc[index-1]['3rd']      
        result['final_score'] = row['Final']
        result['opening_moneyline_home'] = row['Open']
        result['opening_moneyline_away'] = odds.loc[index-1]['Open']
        result['closing_moneyline_home'] = row['Close']
        result['closing_moneyline_away'] = odds.loc[index-1]['Close']
        result['spread'] = row['Puck Line']
        result['away_spread_odds'] = odds.loc[index-1]['Unnamed: 11']
        result['home_spread_odds'] = row['Unnamed: 11']
        result['open_OU'] = row['Open OU']
        result['close_OU'] = row['Close OU']
        result['over_odds'] = row['Unnamed: 13']
        result['under_odds'] = row['Unnamed: 13']
        #print(row['Team'], df.loc[index-1]['Team'])
        #print(result)
        self.final_odds= self.final_odds.append(result, ignore_index=True)
    
  def merge_with_boxscore(self, season):
    season['day'] = season.date.apply(lambda x: datetime.strptime(x, "%Y-%m-%d").day)
    season['month'] = season.date.apply(lambda x: datetime.strptime(x, "%Y-%m-%d").month)

    self.combined = pd.merge(self.final_odds, season, how='left', on=['day', 'month', 'home_team_name', 'away_team_name'])
    self.combined=self.combined.dropna(subset=['home_goals'], axis=0) 

In [332]:
string = "20182019"
df_2018 = HockeySeason(season_string=string)
df_2018.process_box_scores()
df_2018.full_season.head()

url = 'https://raw.githubusercontent.com/d141/epl_predictions/main/NHL_Odds_Data/nhl%20odds%2018.csv'
odds_df = HockeySeasonOdds(link=url, team_names = team_name_dict)
odds_df.merge_with_boxscore(df_2018.full_season)
odds_df.combined.sample()

,date_x,home_team_name,away_team_name,first_period_home,second_period_home,third_period_home,first_period_away,second_period_away,third_period_away,final_score,opening_moneyline_home,closing_moneyline_home,opening_moneyline_away,closing_moneyline_away,spread,away_spread_odds,home_spread_odds,open_OU,close_OU,over_odds,under_odds,day,game_number_x,month,game_number_y,home_goals,away_goals,home_shots,away_shots,home_hits,away_hits,home_ppg,away_ppg,periods_played,date_y,game_day,game_type
893,2/16,Colorado Avalanche,St. Louis Blues,0,0,0,0,0,3,0,103,105,-123,-115,1.5,211,-255,6.0,6.0,-110,-110,16.0,894.0,2.0,894.0,0.0,3.0,32.0,33.0,31.0,16.0,0.0,0.0,3.0,2019-02-16,145,R


In [ ]:
season = True
while season:
  current_date = datetime(month=10, day=3, year=2018)
  this_months_games = combined[combined.month == current_date.month]
  todays_games = this_months_games[this_months_games.day == current_date.day]
  print(todays_games)
  season=False


   game_number_x       home_team_name  ... under_odds  game_number_y
0            1.0  Toronto Maple Leafs  ...       -110            1.0
1            2.0  Washington Capitals  ...       -110            2.0
2            3.0    Vancouver Canucks  ...       -110            3.0
3            4.0      San Jose Sharks  ...       -110            4.0

[4 rows x 35 columns]
